In [1]:
import sys
sys.path.append('/home/rwb/github/skaworkflows')

import logging
import json

from pathlib import Path
import seaborn as sns

logging.basicConfig(level='INFO')
LOGGER = logging.getLogger(__name__)
%pwd 
%cd /home/rwb/Dropbox/University/PhD/experiment_data/chapter3/initial_results/mid/prototype

/home/rwb/Dropbox/University/PhD/experiment_data/chapter3/initial_results/mid/prototype


In [2]:
system_config_path = Path('no_data_mid_sdp_config_prototype_n786_786channels.json')

with open(system_config_path) as fp: 
    system_config = json.load(fp)

flops,compute_bandwidth,memory = system_config['cluster']['system']['resources']['GenericSDP_m0'].values()
print(flops/10**12, compute_bandwidth/10**9)


7.502 14.100652035


In [4]:
# Setup multipler for a given observation
obs_time = 18000
compute_unit = 10**15 # Peta flop
data_unit = 10**6 # per million visibilites 
bytes_per_vis = 12 

In [79]:
# Isolate a single unit of computing 

import pandas as pd
workflows = ['ICAL'] # , 'DPrepA','DPrepB','DPrepC','DPrepD']
low_hpsos = ['hpso13'] #, 'hpso15', 'hpso32']


# structure the data according to | product | workflow | time | type (long-form data) rather than wide form

df = pd.DataFrame()

for wf in workflows: 
    for hpso in low_hpsos:
        hpso_df = pd.DataFrame()
        workflow_data_path = Path(
            f"workflows/workflow_stats/{hpso}_time-28800_channels-786_tel-197_no_data-standard.json_data"
        )
        with open(workflow_data_path) as fp: 
            workflow_data = json.load(fp)

        for product in workflow_data:
            columns = {'HPSO': hpso, 'Product': product, 'Workflow': wf, 'Time (s)': [], 'Type': []}
            curr_product_dict = workflow_data[product]
            if 'total_compute' in workflow_data[product]:
                compute = curr_product_dict['fraction_compute_cost']  * obs_time * compute_unit
                columns['Time (s)'].append(compute / flops)
                columns['Type'].append('compute')
                data = curr_product_dict['fraction_data_cost'] * obs_time * data_unit * bytes_per_vis
                columns['Time (s)'].append(data / compute_bandwidth)
                columns['Type'].append('data')
            tmp_df = pd.DataFrame(columns,)
            hpso_df = hpso_df.append(tmp_df, ignore_index=True)
        df = df.append(hpso_df, ignore_index=True)

In [10]:
# df[df["HPSO"] == 'hpso02b']
df = df.loc[(df!=0).all(1)]
products = set(df["Product"])
products

{'Correct',
 'Degrid',
 'FFT',
 'Flag',
 'Grid',
 'IFFT',
 'Identify Component',
 'Predict',
 'Solve',
 'Source Find',
 'Subtract',
 'Subtract Image Component',
 'UpdateLSM'}

In [80]:

grouped = df.groupby(["HPSO", "Product"])

# for name, group in grouped:
# #     print(name)
# #     print(group)
    
# df_grouped_ex = grouped.get_group(("hpso01", "Correct"))
# mask = df_grouped_ex['HPSO']== df_grouped_ex['HPSO']
# mid_group = df_grouped_ex.groupby(["Type"])

# mid_group.get_group("data")
# for name, group in mid_group:
#     print(name)
#     print(group)

# min_time = min(
#     grouped.get_group(("hpso01", "Correct", "compute")).min()['Time (s)'],
#     grouped.get_group(("hpso01", "Correct", "data")).min()['Time (s)']
# )
# max_time = 
# df_grouped_ex.loc[:, "rank"] = df_grouped_ex.max()["Time (s)"] - df_grouped_ex.min()["Time (s)"]
# df_grouped_ex

hpso_dataframes = {}
for h in low_hpsos:
    df_hpso = df[df["HPSO"] == h]
    grouped = df_hpso.groupby("Product")
    grouped_df = []
    
    for p in products:
        if p not in grouped.groups.keys():
            continue
        df_grouped_ex = grouped.get_group(p)
        tmp_group = df_grouped_ex.groupby(["Type"])
        min_time = min(
            tmp_group.get_group("data").min()["Time (s)"],
            tmp_group.get_group("compute").min()["Time (s)"] 
        )
        max_time = max(
            tmp_group.get_group("data").max()["Time (s)"],
            tmp_group.get_group("compute").max()["Time (s)"] 
        )
        df_grouped_ex.loc[:, "rank"] = max_time-min_time
        grouped_df.append(df_grouped_ex)
    if grouped_df:
        hpso_dataframes[h] = pd.concat(grouped_df, ignore_index=True)

hpso_dataframes['hpso13'] = hpso_dataframes['hpso13'].sort_values(by=['rank','Product'], ascending=False)


/home/rwb/github/topsim/venv/lib/python3.8/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [81]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import CenteredNorm, LogNorm, TwoSlopeNorm, Normalize
from matplotlib import cm
df_comp = df[df['Type']== 'compute'].reset_index(drop=True)
df_data = df[df['Type']== 'data'].reset_index(drop=True)
df_comp['Workflow'] = df_comp['Workflow'].str.replace('ICAL','DICAL')
df_data['Workflow'] = df_data['Workflow'].str.replace('ICAL','DICAL')

# df_data = df_data.sort_values(by=['Workflow','Product'], ascending=[True,False])
# df_comp = df_comp.sort_values(by=['Workflow','Product'], ascending=[True,False])
df_comp.loc[:, "ratio"] = ((np.array(df_data['Time (s)']) / np.array(df_comp['Time (s)'])))

%pwd

plt.ioff()
# df_comp.loc[:, "size"] = [( c/1 if c/1 > 1 else 1/c) for c in df_comp['ratio']]
f, ax = plt.subplots(figsize=(8,5),dpi=300)

# sizes = (df_comp['Time (s)'] - np.mean(df_comp['Time (s)']))/np.std(x)    
colorpalette = 'coolwarm'
# norm = LogNorm(vmin=0.001, vmax=1000)
# midnorm = MidpointNormalize(0, vmax=df_comp['ratio'].max(),vcenter=50)

x = df_comp['Workflow']
y = df_comp['Product']
y2 = np.array(df_comp['ratio'])
# norm = TwoSlopeNorm( vcenter=1, vmin=df_comp['ratio'].min(), vmax=200) # , vmin=df_comp['ratio'].min(), vmax=df_comp['ratio'].max())
# res = ax.scatter(x,y , c=df_comp['ratio'], cmap=colorpalette, norm=norm, s=1000)

rest = ax.scatter(y2, y)
ax.set_ylabel("Algorithm", fontsize=16)
ax.set_xlabel("Ratio (I/O-intensive > 1)", fontsize=16)
ax.yaxis.set_label_coords(-0.15,0.5 )
ax.xaxis.set_label_coords(0.5,-0.1)
ax.spines['left'].set_position(('data', 1))
ax.spines['right'].set_visible(False)
ax.set_xscale('log')
# cb = f.colorbar(res)

# cb.ax.set_ylabel('Ratio (I/O : FLOPs)', rotation=270)
# cb.ax.yaxis.set_label_coords(4,0.5)
ax.set_title("Runtime comparison of SKA Workflow algorithms as a ratio of I/O to FLOPs: HPSO13 Observation", pad=25)
# plt.show()
plt.savefig("RuntimeComparisons_13.png", facecolor="white", transparent=False, bbox_inches="tight")
# df_comp, df_data
# 66 	HPSO01 	Correct 	DPrepD 	436.435911 	compute 	450.602210 	3.703428e

/tmp/ipykernel_54786/2327265640.py:12: RuntimeWarning: invalid value encountered in true_divide
  df_comp.loc[:, "ratio"] = ((np.array(df_data['Time (s)']) / np.array(df_comp['Time (s)'])))


In [ ]:
plt.style.use('_mpl-gallery')

# make data
np.random.seed(1)
x = 4 + np.random.normal(0, 1.5, 200)

# plot:
fig, ax = plt.subplots()

ax.hist(x, bins=8, linewidth=0.5, edgecolor="white")

ax.set(xlim=(0, 8), xticks=np.arange(1, 8),
       ylim=(0, 56), yticks=np.linspace(0, 56, 9))

plt.show()


In [ ]:
df_hpso01 = hpso_dataframes['hpso02a']
df_compute = df_hpso01[df_hpso01['Type'] == 'compute']
df_data = df_hpso01[df_hpso01['Type'] == 'data']

fix, ax = plt.subplots(figsize=(8,8))

bins = np.arange(0, 2000, 50)

ax.hist(df_compute['Time (s)'], bins, histtype='bar')

df_compute['Time (s)']
df_data['Time (s)']

df_max = pd.DataFrame({'compute': np.array(df_compute['Time (s)']), 'data': np.array(df_data['Time (s)'])}).max(axis=1)

# sorted(df_compute['Time (s)'])
ax.hist(df_max, bins, histtype='bar')
a =ax.set_xticks(ticks=bins,minor=True)

df_max